#**OBTAINING THE DATASET**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/LineMOD_DATASET/Linemod_preprocessed.zip

Streaming output truncated to the last 5000 lines.
  inflating: Linemod_preprocessed/segnet_results/11_label/0206_label.png  
  inflating: Linemod_preprocessed/segnet_results/11_label/0207_label.png  
  inflating: Linemod_preprocessed/segnet_results/11_label/0208_label.png  
  inflating: Linemod_preprocessed/segnet_results/11_label/0209_label.png  
  inflating: Linemod_preprocessed/segnet_results/11_label/0210_label.png  
  inflating: Linemod_preprocessed/segnet_results/11_label/0211_label.png  
  inflating: Linemod_preprocessed/segnet_results/11_label/0212_label.png  
  inflating: Linemod_preprocessed/segnet_results/11_label/0213_label.png  
  inflating: Linemod_preprocessed/segnet_results/11_label/0214_label.png  
  inflating: Linemod_preprocessed/segnet_results/11_label/0215_label.png  
  inflating: Linemod_preprocessed/segnet_results/11_label/0216_label.png  
  inflating: Linemod_preprocessed/segnet_results/11_label/0218_label.png  
  inflating: Linemod_preprocessed/segnet_results/

In [ ]:
!pip install pcl
!pip install open3d
!pip install plotly
!pip install pyyaml

#**DATASET AND DATALOADER**

In [ ]:
import os
import yaml
import numpy as np
import torch
import open3d as o3d
from torch.utils.data import Dataset
from PIL import Image
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import open3d as o3d
import numpy as np
import plotly.graph_objects as go
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 127.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 98.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 95.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [ ]:
#!rm -rf /content/YOLO_LINEMOD  #for deleting the directory on google colab


In [ ]:
import os
import cv2
import yaml
import random
from tqdm import tqdm
from sklearn.model_selection import train_test_split

# ==== Paths ====
LINEMOD_PATH = '/content/Linemod_preprocessed/data'
YOLO_PATH = '/content/YOLO_LINEMOD'
IMAGE_WIDTH = 640
IMAGE_HEIGHT = 480

CLASS_NAMES = ['ape', 'benchvise', 'bowl', 'can', 'cat', 'cup', 'driller', 'duck',
               'glue', 'holepuncher', 'iron', 'lamp', 'phone', 'cam', 'eggbox']

# ==== Create output folders ====
for split in ['train', 'val', 'test']:
    os.makedirs(f'{YOLO_PATH}/images/{split}', exist_ok=True)
    os.makedirs(f'{YOLO_PATH}/labels/{split}', exist_ok=True)

def convert_bbox_to_yolo(x, y, w, h, img_w, img_h):
    x_center = (x + w / 2) / img_w
    y_center = (y + h / 2) / img_h
    w_norm = w / img_w
    h_norm = h / img_h
    return x_center, y_center, w_norm, h_norm

# ==== Process each class ====
for obj_id, class_name in enumerate(CLASS_NAMES, start=1):
    class_folder = f'{obj_id:02d}'
    folder_path = os.path.join(LINEMOD_PATH, class_folder)

    train_txt_path = os.path.join(folder_path, 'train.txt')
    test_txt_path = os.path.join(folder_path, 'test.txt')
    gt_path = os.path.join(folder_path, 'gt.yml')

    if not os.path.exists(train_txt_path) or not os.path.exists(test_txt_path) or not os.path.exists(gt_path):
        print(f"Skipping {class_name} (missing files)")
        continue

    train_ids_full = [int(line.strip()) for line in open(train_txt_path)]
    test_ids = set(int(line.strip()) for line in open(test_txt_path))

    # Split train into train and val
    train_ids, val_ids = train_test_split(train_ids_full, test_size=0.2, random_state=42)

    gt = yaml.safe_load(open(gt_path))

    for img_id_str in tqdm(gt.keys(), desc=f'Class {class_name}'):
        img_id = int(img_id_str)
        img_name = f'{img_id:04d}.png'
        image_path = os.path.join(folder_path, 'rgb', img_name)
        if not os.path.exists(image_path):
            continue

        # Determine subset
        if img_id in train_ids:
            subset = 'train'
        elif img_id in val_ids:
            subset = 'val'
        elif img_id in test_ids:
            subset = 'test'
        else:
            continue

        out_image_path = os.path.join(YOLO_PATH, f'images/{subset}', f'{class_folder}_{img_name}')
        out_label_path = os.path.join(YOLO_PATH, f'labels/{subset}', f'{class_folder}_{img_name.replace(".png", ".txt")}')

        image = cv2.imread(image_path)
        cv2.imwrite(out_image_path, image)

        with open(out_label_path, 'w') as f:
            for obj in gt[img_id]:
                if obj['obj_id'] != obj_id:
                    continue
                x, y, w, h = obj['obj_bb']
                yolo_box = convert_bbox_to_yolo(x, y, w, h, image.shape[1], image.shape[0])
                label_line = f"{obj_id - 1} {' '.join(f'{v:.6f}' for v in yolo_box)}\n"
                f.write(label_line)

# ==== Create data.yaml (train/val only) ====
with open(os.path.join(YOLO_PATH, 'data.yaml'), 'w') as f:
    f.write(f'''train: {YOLO_PATH}/images/train
val: {YOLO_PATH}/images/val
nc: {len(CLASS_NAMES)}
names: {CLASS_NAMES}
''')


Class benchvise: 100%|██████████| 1214/1214 [00:37<00:00, 32.00it/s]


Skipping bowl (missing files)


Class cup: 100%|██████████| 1179/1179 [00:36<00:00, 32.00it/s]


Skipping driller (missing files)


Class eggbox: 100%|██████████| 1243/1243 [00:38<00:00, 32.45it/s]


In [ ]:
# !rm -rf /content/runs/detect/linemod_yolo_v8n22  #for deleting the directory on google colab
#

In [ ]:
from google.colab import drive
import os
import shutil
import json
from ultralytics import YOLO
import pandas as pd
import glob
# ==== 1. Mount Google Drive ====
drive.mount('/content/drive')

# ==== 2. Set up paths ====
run_name = "linemod_yolo_v8n"
save_dir = f"/content/drive/MyDrive/yolo_models/{run_name}"
os.makedirs(save_dir, exist_ok=True)

# ==== 3. Load or train the model ====
checkpoint_path = f"runs/detect/{run_name}/weights/last.pt"

if os.path.exists(checkpoint_path):
    print("📦 Resuming from previous checkpoint...")
    model = YOLO(checkpoint_path)
    model.train(resume=True)
else:
    print("🚀 Starting new training...")
    model = YOLO("yolov8n.pt")
    model.train(
        data="/content/YOLO_LINEMOD/data.yaml",  # Contains only train/val
        epochs=300,
        imgsz=640,
        batch=16,
        name=run_name,
        device=0  # GPU (or "cpu" if needed)
    )

# ==== 4. Evaluate on validation set ====
print("\n🔍 Validation mAP evaluation (val set used in training config)...")
val_metrics = model.val()
print(f"\nmAP@0.5 (val): {val_metrics.box.map50:.4f}")
print(f"mAP@0.5:0.95 (val): {val_metrics.box.map:.4f}")

# ==== 5. Export model to ONNX format ====
print("\n📦 Exporting model to ONNX format...")
export_result = model.export(format="onnx")
onnx_path = export_result[0] if isinstance(export_result, tuple) else export_result
print(onnx_path)
# Save ONNX export
if os.path.exists(onnx_path):
    shutil.copy(onnx_path, save_dir)

# ==== 6. Save outputs to Google Drive ====
print("\n💾 Saving outputs to Google Drive...")

# Find latest run directory matching the run_name prefix
run_dirs = sorted(glob.glob(f"runs/detect/{run_name}*"), key=os.path.getmtime)
latest_run_dir = run_dirs[-1]  # Most recently modified run
weights_dir = os.path.join(latest_run_dir, "weights")
train_results_dir = latest_run_dir
print(train_results_dir)
test_predictions_dir = f"runs/detect/test_predictions"


# Save weights
for file_name in ["best.pt", "last.pt"]:
    src = os.path.join(weights_dir, file_name)
    if os.path.exists(src):
        shutil.copy(src, save_dir)



# Save training results
for file_name in ["results.png", "confusion_matrix.png", "results.csv"]:
    src = os.path.join(train_results_dir, file_name)
    if os.path.exists(src):
        shutil.copy(src, save_dir)

# ==== 7. Save validation metrics ====
val_metrics_dict = {
    "mAP@0.5 (val)": float(val_metrics.box.map50),
    "mAP@0.5:0.95 (val)": float(val_metrics.box.map),
    "per_class_mAP": [float(x) for x in val_metrics.box.maps]
}
with open(f"{save_dir}/val_metrics.json", "w") as f:
    json.dump(val_metrics_dict, f, indent=4)



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🚀 Starting new training...
Ultralytics 8.3.123 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/YOLO_LINEMOD/data.yaml, epochs=300, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=8, project=None, name=linemod_yolo_v8n2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, reti

train: Scanning /content/YOLO_LINEMOD/labels/train... 1893 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1893/1893 [00:05<00:00, 360.93it/s]


train: New cache created: /content/YOLO_LINEMOD/labels/train.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2956.4±1963.2 MB/s, size: 509.9 KB)


val: Scanning /content/YOLO_LINEMOD/labels/val... 480 images, 0 backgrounds, 0 corrupt: 100%|██████████| 480/480 [00:01<00:00, 439.98it/s]

val: New cache created: /content/YOLO_LINEMOD/labels/val.cache


...


300 epochs completed in 1.660 hours.
Optimizer stripped from runs/detect/linemod_yolo_v8n2/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/linemod_yolo_v8n2/weights/best.pt, 6.3MB

Validating runs/detect/linemod_yolo_v8n2/weights/best.pt...
Ultralytics 8.3.123 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,008,573 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:02<00:00,  5.81it/s]


                   all        480        480      0.998          1      0.995      0.955
                   ape         38         38      0.999          1      0.995      0.947
             benchvise         37         37      0.998          1      0.995      0.943
                   can         37         37      0.998          1      0.995      0.966
                   cat         36         36      0.998          1      0.995      0.973
                   cup         36         36          1          1      0.995       0.93
                  duck         36         36      0.998          1      0.995      0.971
                  glue         38         38      0.999          1      0.995      0.941
           holepuncher         38         38      0.998          1      0.995      0.981
                  iron         37         37      0.999          1      0.995      0.927
                  lamp         38         38      0.998          1      0.995      0.936
                 phon

val: Scanning /content/YOLO_LINEMOD/labels/val.cache... 480 images, 0 backgrounds, 0 corrupt: 100%|██████████| 480/480 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 30/30 [00:03<00:00,  8.32it/s]


                   all        480        480      0.998          1      0.995      0.954
                   ape         38         38      0.999          1      0.995      0.952
             benchvise         37         37      0.998          1      0.995      0.941
                   can         37         37      0.998          1      0.995      0.968
                   cat         36         36      0.998          1      0.995       0.97
                   cup         36         36          1          1      0.995      0.933
                  duck         36         36      0.998          1      0.995      0.967
                  glue         38         38      0.999          1      0.995      0.938
           holepuncher         38         38      0.998          1      0.995      0.981
                  iron         37         37      0.999          1      0.995      0.923
                  lamp         38         38      0.998          1      0.995      0.935
                 phon

In [ ]:
import shutil
import os

# Target location in Google Drive
drive_backup_path = "/content/drive/MyDrive/yolo_models/runs_backup"

# Source directory
runs_dir = "/content/runs"

# Copy the entire runs folder to Google Drive
print("\n💾 Copying full /runs directory to Google Drive...")
shutil.copytree(runs_dir, drive_backup_path, dirs_exist_ok=True)
print(f"✅ Full /runs folder saved to: {drive_backup_path}")



💾 Copying full /runs directory to Google Drive...
✅ Full /runs folder saved to: /content/drive/MyDrive/yolo_models/runs_backup


In [ ]:
import glob
import os
from ultralytics import YOLO

run_name = "linemod_yolo_v8n"
split = "test"

# # Find latest matching run directory
# run_dirs = sorted(glob.glob(f"runs/detect/{run_name}*"), key=os.path.getmtime)
# latest_run_dir = run_dirs[-1]  # Most recently modified
# weights_path = os.path.join(latest_run_dir, "weights", "best.pt")  # or 'last.pt'

# ==== 8. Run inference on the test set ====
print("\n🖼️ Predicting on the separate test set...")
test_results = model.predict(
    source="/content/YOLO_LINEMOD/images/test",
    conf=0.25,
    save=True,
    save_txt=True,
    imgsz=640,
    name="test_predictions"
)
detections = []

for img_result in test_results:
    name = os.path.basename(img_result.path)
    for box in img_result.boxes.data:
        cls, conf, x1, y1, x2, y2 = int(box[5]), float(box[4]), float(box[0]), float(box[1]), float(box[2]), float(box[3])
        detections.append({
            'filename': name,
            'split': split,
            'class_id': cls,
            'conf': conf,
            'x1': x1, 'y1': y1, 'x2': x2, 'y2': y2
        })

df = pd.DataFrame(detections)
df.to_csv(f"/content/YOLO_LINEMOD/detections_{split}.csv", index=False)
# Save predictions (test set)
if os.path.exists(test_predictions_dir):
    shutil.copytree(test_predictions_dir, os.path.join(save_dir, "test_predictions"), dirs_exist_ok=True)

print(f"\n✅ All outputs saved to: {save_dir}")


Streaming output truncated to the last 5000 lines.
image 8413/13407 /content/YOLO_LINEMOD/images/test/11_0181.png: 480x640 1 iron, 6.4ms
image 8414/13407 /content/YOLO_LINEMOD/images/test/11_0182.png: 480x640 1 iron, 6.3ms
image 8415/13407 /content/YOLO_LINEMOD/images/test/11_0183.png: 480x640 1 iron, 6.5ms
image 8416/13407 /content/YOLO_LINEMOD/images/test/11_0184.png: 480x640 1 iron, 7.1ms
image 8417/13407 /content/YOLO_LINEMOD/images/test/11_0185.png: 480x640 1 iron, 6.3ms
image 8418/13407 /content/YOLO_LINEMOD/images/test/11_0186.png: 480x640 1 iron, 6.4ms
image 8419/13407 /content/YOLO_LINEMOD/images/test/11_0187.png: 480x640 1 iron, 6.3ms
image 8420/13407 /content/YOLO_LINEMOD/images/test/11_0188.png: 480x640 1 iron, 6.1ms
image 8421/13407 /content/YOLO_LINEMOD/images/test/11_0189.png: 480x640 1 iron, 6.1ms
image 8422/13407 /content/YOLO_LINEMOD/images/test/11_0190.png: 480x640 1 iron, 6.1ms
image 8423/13407 /content/YOLO_LINEMOD/images/test/11_0191.png: 480x640 1 iron, 6.1ms
ima

In [ ]:
# import os
# import json
# from tqdm import tqdm
# from pycocotools.coco import COCO
# from pycocotools.cocoeval import COCOeval
# from pathlib import Path

# # ==== 1. Setup Paths ====
# test_images_dir = "/content/YOLO_LINEMOD/images/test"
# test_labels_dir = "/content/YOLO_LINEMOD/labels/test"
# predictions_dir = "runs/detect/test_predictions/labels"
# class_names = ['ape', 'benchvise', 'bowl', 'can', 'cat', 'cup', 'driller', 'duck',
#                'glue', 'holepuncher', 'iron', 'lamp', 'phone', 'cam', 'eggbox']

# # ==== 2. Build COCO-style ground truth ====
# coco_gt = {
#     "images": [],
#     "annotations": [],
#     "categories": [{"id": i, "name": name} for i, name in enumerate(class_names)]
# }

# ann_id = 1
# image_id_map = {}  # To map filename to image_id

# for idx, file in enumerate(sorted(os.listdir(test_labels_dir))):
#     if not file.endswith(".txt"):
#         continue
#     img_name = file.replace(".txt", ".png")
#     img_path = os.path.join(test_images_dir, img_name)
#     if not os.path.exists(img_path):
#         continue

#     image_id = idx + 1
#     image_id_map[img_name] = image_id

#     coco_gt["images"].append({
#         "id": image_id,
#         "file_name": img_name,
#         "width": 640,
#         "height": 480
#     })

#     with open(os.path.join(test_labels_dir, file), "r") as f:
#         for line in f:
#             parts = list(map(float, line.strip().split()))
#             class_id, x_center, y_center, w, h = parts
#             x = (x_center - w / 2) * 640
#             y = (y_center - h / 2) * 480
#             w *= 640
#             h *= 480
#             coco_gt["annotations"].append({
#                 "id": ann_id,
#                 "image_id": image_id,
#                 "category_id": int(class_id),
#                 "bbox": [x, y, w, h],
#                 "area": w * h,
#                 "iscrowd": 0
#             })
#             ann_id += 1

# with open("gt_coco.json", "w") as f:
#     json.dump(coco_gt, f)

# # ==== 3. Build COCO-style predictions ====
# coco_dt = []

# for file in sorted(os.listdir(predictions_dir)):
#     if not file.endswith(".txt"):
#         continue
#     img_name = file.replace(".txt", ".png")
#     image_id = image_id_map.get(img_name)
#     if image_id is None:
#         continue

#     with open(os.path.join(predictions_dir, file), "r") as f:
#         for line in f:
#             parts = list(map(float, line.strip().split()))
#             class_id, x_center, y_center, w, h, conf = parts
#             x = (x_center - w / 2) * 640
#             y = (y_center - h / 2) * 480
#             w *= 640
#             h *= 480
#             coco_dt.append({
#                 "image_id": image_id,
#                 "category_id": int(class_id),
#                 "bbox": [x, y, w, h],
#                 "score": conf
#             })

# with open("pred_coco.json", "w") as f:
#     json.dump(coco_dt, f)

# # ==== 4. Run COCO Evaluation ====
# coco_gt = COCO("gt_coco.json")
# coco_dt = coco_gt.loadRes("pred_coco.json")
# coco_eval = COCOeval(coco_gt, coco_dt, "bbox")
# coco_eval.evaluate()
# coco_eval.accumulate()
# coco_eval.summarize()
